In [1]:
import multiprocessing as mp
import psutil 
from rosette import Rosette
import pyvista as pv
import concurrent.futures
from tqdm import tqdm
from functools import partial
from itertools import repeat
# from multiprocessing import Pool, freeze_support
from multiprocess import Pool, freeze_support
import numpy as np

In [ ]:
mp.cpu_count()

In [ ]:
n_core_physical = psutil.cpu_count(logical = False)
n_core_logical = psutil.cpu_count(logical = True)
print("# physical cores = ", n_core_physical)
print("# logical cores = ", n_core_logical)

In [ ]:
# create a single rosette

# set geoemetric parameters
a = 0.5 # half max length across basal face
c =  2.0 # half max length across prism face
r0 = 1.0 # radius of center sphere
h0 = 0.25 # penetration depth of bullets
hp = 0.75 # heights of pyramid of bullets
n_arms = 8 # number of bullet arms
test = Rosette(a, c, r0, h0, hp, n_arms)
test.unify_mesh()
test_mesh = test.model

In [ ]:
# save test mesh as stl file
test_mesh.save('test_mesh.stl')

In [ ]:
%%time
# create rosette 100 times 
# set geoemetric parameters
a = 0.5 # half max length across basal face
c =  2.0 # half max length across prism face
r0 = 1.0 # radius of center sphere
h0 = 0.25 # penetration depth of bullets
hp = 0.75 # heights of pyramid of bullets
n_arms = 12 # number of bullet arms
N = 1_000
for i in tqdm(range(N)):
    test = Rosette(a, c, r0, h0, hp, n_arms)

In [ ]:
%%time
# create rosette 100 times, in parallel
n_cores = psutil.cpu_count(logical = False)
# set geoemetric parameters
a = 0.5 # half max length across basal face
c =  2.0 # half max length across prism face
r0 = 1.0 # radius of center sphere
h0 = 0.25 # penetration depth of bullets
hp = 0.75 # heights of pyramid of bullets
n_arms = 12 # number of bullet arms
N = 1_000
def create_rosette(N):
    test = Rosette(a, c, r0, h0, hp, n_arms)

def main():
    with Pool(n_cores) as pool:
        pool.map(create_rosette, range(N))

if __name__=="__main__":
    main()


In [ ]:
a = 0.20 # half max length across basal face
c =  1.0 # half max length across prism face
r0 = 1.0 # radius of center sphere
h0 = 0.25 # penetration depth of bullets
hp = 0.75 # heights of pyramid of bullets
n_arms = 9 # number of bullet arms

ros = Rosette(a, c, r0, h0, hp, n_arms)
pl = ros.plot()
pl.show()


# ros.unify_mesh()
# ros_file_name = f'ros_n{n_arms:n}_a{a:.2f}_c{c:.2f}_0.stl'
# ros_file_path = save_path + '/' + ros_file_name
# ros.model.save(ros_file_path)

In [ ]:
a = 0.70 # half max length across basal face
c =  1.0 # half max length across prism face
r0 = 1.0 # radius of center sphere
h0 = 0.25 # penetration depth of bullets
hp = 0.75 # heights of pyramid of bullets
n_arms = 9 # number of bullet arms

def generate_rosette(a, c, r0, h0, hp, n_arms):
    save_path = '/Users/josephko/research/ice_renders/20230329'
    basal_min, basal_max = 0.75, 1.25
    prism_min, prism_max = 0.5, 1.5
    scaling_range = [basal_min, basal_max, prism_min, prism_max]
    solid_angle = 15
    ros = Rosette(a, c, r0, h0, hp, n_arms)
    ros.unify_mesh()
    ros_file_name = f'ros_n{n_arms:n}_a{a:.2f}_c{c:.2f}_0.stl'
    ros_file_path = save_path + '/' + ros_file_name
    ros.model.save(ros_file_path)
    # for i in range(5): # create 9 random versions
    #     ros_rand = ros.randomize_bullets(scaling=scaling_range, location=solid_angle, inplace=False)
    #     ros_rand.unify_mesh()
    #     ros_rand_file_name = f'ros_n{n_arms:n}_a{a:.2f}_c{c:.2f}_{i+1:n}.stl'
    #     ros_rand_file_path = save_path + '/' + ros_rand_file_name
    #     # ros_rand.model.save(ros_rand_file_path)

generate_rosette(a, c, r0, h0, hp, n_arms)

In [2]:
from multiprocess import get_context
from multiprocessing import set_start_method
set_start_method('fork')

In [3]:
%%time
def set_args(n_arr):
    # create argument array for use with multi-parameter function
    # r0, h0, hp set constant for this test to reduce N
    a_arr = np.linspace(0.1, 0.5, n_arr)
    c_arr = np.linspace(1.0, 4.0, n_arr)
    n_arms_arr = np.arange(1, 9, dtype=int)
    r0 = 1.0 # radius of center sphere
    h0 = 0.25 # penetration depth of bullets
    hp = 0.75 # heights of pyramid of bullets
    args_list = []
    for a in a_arr:
        for c in c_arr:
            for n_arms in n_arms_arr:
                args = (a, c, r0, h0, hp, n_arms)
                args_list.append(args)
    return args_list

def generate_rosette(a, c, r0, h0, hp, n_arms):
    save_path = '/Users/josephko/research/ice_renders/20230329'
    basal_min, basal_max = 0.75, 1.25
    prism_min, prism_max = 0.5, 1.5
    scaling_range = [basal_min, basal_max, prism_min, prism_max]
    solid_angle = 15
    ros = Rosette(a, c, r0, h0, hp, n_arms)
    ros.unify_mesh()
    ros_file_name = f'ros_n{n_arms:n}_a{a:.2f}_c{c:.2f}_0.stl'
    ros_file_path = save_path + '/' + ros_file_name
    ros.model.save(ros_file_path)
    for i in range(9): # create randomized versions
        ros_rand = ros.randomize_bullets(scaling=scaling_range, location=solid_angle, inplace=False)
        ros_rand.unify_mesh()
        ros_rand_file_name = f'ros_n{n_arms:n}_a{a:.2f}_c{c:.2f}_{i+1:n}.stl'
        ros_rand_file_path = save_path + '/' + ros_rand_file_name
        ros_rand.model.save(ros_rand_file_path)

# def main():
#     args_list = set_args(5)
#     n_cores = psutil.cpu_count(logical = False)
#     with Pool(n_cores) as pool:
#         pool.starmap(generate_rosette, args_list)
#         pool.close()
def main():
    args_list = set_args(5)
    n_cores = psutil.cpu_count(logical = False)
    pool = Pool(n_cores)
    pool.starmap(generate_rosette, args_list)
    pool.close()


if __name__=="__main__":
    freeze_support()
    main()

CPU times: user 36.4 ms, sys: 25.1 ms, total: 61.5 ms
Wall time: 1min 1s


In [11]:
%%time
# SERIAL VERSION OF CODE ABOVE
def set_args(n_arr):
    # create argument array for use with multi-parameter function
    # r0, h0, hp set constant for this test to reduce N
    a_arr = np.linspace(0.1, 0.5, n_arr)
    c_arr = np.linspace(1.0, 4.0, n_arr)
    n_arms_arr = np.arange(1, 9, dtype=int)
    r0 = 1.0 # radius of center sphere
    h0 = 0.25 # penetration depth of bullets
    hp = 0.75 # heights of pyramid of bullets
    args_list = []
    for a in a_arr:
        for c in c_arr:
            for n_arms in n_arms_arr:
                args = (a, c, r0, h0, hp, n_arms)
                args_list.append(args)
    return args_list

def generate_rosette(a, c, r0, h0, hp, n_arms):
    save_path = '/Users/josephko/research/ice_renders/20230329_serial'
    basal_min, basal_max = 0.75, 1.25
    prism_min, prism_max = 0.5, 1.5
    scaling_range = [basal_min, basal_max, prism_min, prism_max]
    solid_angle = 15
    ros = Rosette(a, c, r0, h0, hp, n_arms)
    ros.unify_mesh()
    ros_file_name = f'ros_n{n_arms:n}_a{a:.2f}_c{c:.2f}_0.stl'
    ros_file_path = save_path + '/' + ros_file_name
    ros.model.save(ros_file_path)
    for i in range(9): # create randomized versions
        ros_rand = ros.randomize_bullets(scaling=scaling_range, location=solid_angle, inplace=False)
        ros_rand.unify_mesh()
        ros_rand_file_name = f'ros_n{n_arms:n}_a{a:.2f}_c{c:.2f}_{i+1:n}.stl'
        ros_rand_file_path = save_path + '/' + ros_rand_file_name
        ros_rand.model.save(ros_rand_file_path)

# def main():
#     args_list = set_args(5)
#     n_cores = psutil.cpu_count(logical = False)
#     with Pool(n_cores) as pool:
#         pool.starmap(generate_rosette, args_list)
#         pool.close()
def main():
    args_list = set_args(5)
    for args in args_list:
        generate_rosette(args[0], args[1], args[2], args[3], args[4], args[5])


if __name__=="__main__":
    # freeze_support()
    main()

ERROR:root:Algorithm vtkIntersectionPolyDataFilter (0x127efce10) returned failure for request: vtkInformation (0x127eeffb0)
2023-03-30 01:13:23.367 ( 969.842s) [          ADD887]vtkIntersectionPolyData:1593  WARN| vtkIntersectionPolyDataFilter (0x127efce10): No cell with correct orientation found
2023-03-30 01:13:23.367 ( 969.843s) [          ADD887]       vtkExecutive.cxx:741    ERR| vtkCompositeDataPipeline (0x127eff8e0): Algorithm vtkIntersectionPolyDataFilter (0x127efce10) returned failure for request: vtkInformation (0x127eeffb0)
  Debug: Off
  Modified Time: 1620221094
  Reference Count: 1
  Registered Events: (none)
  Request: REQUEST_DATA
  FROM_OUTPUT_PORT: 0
  ALGORITHM_AFTER_FORWARD: 1
  FORWARD_DIRECTION: 0


ERROR:root:Algorithm vtkBooleanOperationPolyDataFilter (0x127edd6a0) returned failure for request: vtkInformation (0x127eed400)
2023-03-30 01:13:23.368 ( 969.843s) [          ADD887]       vtkExecutive.cxx:741    ERR| vtkCompositeDataPipeline (0x127ef7010): Algorithm v

KeyboardInterrupt: 

In [8]:
def set_args(n_arr):
    # create argument array for use with multi-parameter function
    # r0, h0, hp set constant for this test to reduce N
    a_arr = np.linspace(0.1, 0.5, n_arr)
    c_arr = np.linspace(1.0, 4.0, n_arr)
    n_arms_arr = np.arange(1, 9, dtype=int)
    r0 = 1.0 # radius of center sphere
    h0 = 0.25 # penetration depth of bullets
    hp = 0.75 # heights of pyramid of bullets
    args_list = []
    for a in a_arr:
        for c in c_arr:
            for n_arms in n_arms_arr:
                args = (a, c, r0, h0, hp, n_arms)
                args_list.append(args)
    return args_list

def generate_rosette(a, c, r0, h0, hp, n_arms):
    save_path = '/Users/josephko/research/ice_renders/20230329_serial'
    basal_min, basal_max = 0.75, 1.25
    prism_min, prism_max = 0.5, 1.5
    scaling_range = [basal_min, basal_max, prism_min, prism_max]
    solid_angle = 15
    ros = Rosette(a, c, r0, h0, hp, n_arms)
    ros.unify_mesh()
    ros_file_name = f'ros_n{n_arms:n}_a{a:.2f}_c{c:.2f}_0.stl'
    ros_file_path = save_path + '/' + ros_file_name
    ros.model.save(ros_file_path)
    for i in range(9): # create randomized versions
        ros_rand = ros.randomize_bullets(scaling=scaling_range, location=solid_angle, inplace=False)
        ros_rand.unify_mesh()
        ros_rand_file_name = f'ros_n{n_arms:n}_a{a:.2f}_c{c:.2f}_{i+1:n}.stl'
        ros_rand_file_path = save_path + '/' + ros_rand_file_name
        ros_rand.model.save(ros_rand_file_path)

args_list = set_args(2)

# for args in args_list:
#     print(args)

0.1


In [ ]:
n_arms_arr = np.arange(1, 8, dtype=int)
n_arms_arr

In [ ]:
from functools import partial
from itertools import repeat
# from multiprocessing import Pool, freeze_support
from multiprocess import Pool, freeze_support

def func(a, b):
    return a + b

def main():
    a_args = [1,2,3]
    second_arg = 1
    with Pool() as pool:
        L = pool.starmap(func, [(1, 1), (2, 1), (3, 1)])
        M = pool.starmap(func, zip(a_args, repeat(second_arg)))
        N = pool.map(partial(func, b=second_arg), a_args)
        assert L == M == N

if __name__=="__main__":
    freeze_support()
    main()